In [1]:
import joblib
import json
import pandas
import warnings
import numpy

import sklearn.neural_network
import sklearn.model_selection
import sklearn.exceptions

warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)

In [2]:
data_train = pandas.read_csv('../data/features.train.csv').set_index('id')
data_train

,fnlwgt,age-group,education-num,capitalgain,capitalloss,hoursperweek,workclass_self-emp-not-inc,workclass_private,workclass_self-emp-inc,workclass_local-gov,...,native-country_honduras,native-country_thailand,native-country_laos,native-country_peru,native-country_holand-netherlands,native-country_haiti,native-country_scotland,sex_male,sex_female,label
id,,,,,,,,,,,,,,,,,,,,,
2103,1.395470,2.0,14.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
14649,-0.873898,0.0,8.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7379,-0.592186,2.0,8.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24479,-1.349595,2.0,9.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19532,2.041210,3.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,-1.405521,0.0,9.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2192,-0.821560,2.0,9.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8250,1.210247,0.0,8.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
model = sklearn.neural_network.MLPClassifier(
    solver='sgd', 
    activation='logistic',
    alpha=0.0,  
    batch_size=32,
    learning_rate='constant',
    learning_rate_init=0.1,
    max_iter=1000,
    momentum=0.0, 
    nesterovs_momentum=False, 
    validation_fraction=0.0,
    shuffle=True,
    random_state=0
)

In [4]:
search_parameters = {
    'alpha': [0.001, 0.003, 0.002, 0.005, 0.007, 0.009, 0.1, 0.3, 0.5, 0.7, 0.9],
    'hidden_layer_sizes':[(20, 10), (64, 32, 16), (128, 64, 32), (63, 32, 16, 8), (128, 64, 32, 16)]
}

In [5]:
optimizer = sklearn.model_selection.GridSearchCV(
    estimator=model, 
    param_grid=search_parameters,
    n_jobs=-1,
    verbose=2
)

In [6]:
optimizer.fit(
    data_train.drop(['label'], axis='columns'),
    data_train['label'],
)

Fitting 5 folds for each of 55 candidates, totalling 275 fits


GridSearchCV(estimator=MLPClassifier(activation='logistic', alpha=0.0,
                                     batch_size=32, learning_rate_init=0.1,
                                     max_iter=1000, momentum=0.0,
                                     nesterovs_momentum=False, random_state=0,
                                     solver='sgd', validation_fraction=0.0),
             n_jobs=-1,
             param_grid={'alpha': [0.001, 0.003, 0.002, 0.005, 0.007, 0.009,
                                   0.1, 0.3, 0.5, 0.7, 0.9],
                         'hidden_layer_sizes': [(20, 10), (64, 32, 16),
                                                (128, 64, 32), (63, 32, 16, 8),
                                                (128, 64, 32, 16)]},
             verbose=2)

In [7]:
cv_results = pandas.DataFrame(optimizer.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,60.139355,9.023936,0.017874,0.000682,0.001,"(20, 10)","{'alpha': 0.001, 'hidden_layer_sizes': (20, 10)}",0.817225,0.824242,0.827432,0.822010,0.819139,0.822010,0.003620,5
1,51.747528,14.558503,0.028258,0.001905,0.001,"(64, 32, 16)","{'alpha': 0.001, 'hidden_layer_sizes': (64, 32...",0.808293,0.825199,0.814992,0.812440,0.822329,0.816651,0.006255,18
2,63.556352,12.139283,0.038616,0.002949,0.001,"(128, 64, 32)","{'alpha': 0.001, 'hidden_layer_sizes': (128, 6...",0.817544,0.830303,0.822329,0.823923,0.820734,0.822967,0.004232,2
3,44.628472,43.501111,0.025269,0.005047,0.001,"(63, 32, 16, 8)","{'alpha': 0.001, 'hidden_layer_sizes': (63, 32...",0.580542,0.815630,0.580542,0.816587,0.580861,0.674833,0.115352,31
4,129.356023,21.904520,0.039742,0.001457,0.001,"(128, 64, 32, 16)","{'alpha': 0.001, 'hidden_layer_sizes': (128, 6...",0.818501,0.826475,0.827751,0.817863,0.823604,0.822839,0.004038,3
5,36.613034,8.293575,0.011109,0.000416,0.003,"(20, 10)","{'alpha': 0.003, 'hidden_layer_sizes': (20, 10)}",0.819139,0.819777,0.809250,0.816587,0.818820,0.816715,0.003884,17
6,39.168094,6.505639,0.021390,0.001526,0.003,"(64, 32, 16)","{'alpha': 0.003, 'hidden_layer_sizes': (64, 32...",0.809250,0.827113,0.812759,0.812440,0.820415,0.816396,0.006495,19
7,65.288633,11.542435,0.036058,0.001613,0.003,"(128, 64, 32)","{'alpha': 0.003, 'hidden_layer_sizes': (128, 6...",0.817225,0.828070,0.820734,0.823923,0.817225,0.821435,0.004151,6
8,54.815594,39.396379,0.021721,0.000761,0.003,"(63, 32, 16, 8)","{'alpha': 0.003, 'hidden_layer_sizes': (63, 32...",0.580542,0.825837,0.825518,0.814035,0.580861,0.725359,0.118189,30
9,83.307469,18.879284,0.037702,0.004270,0.003,"(128, 64, 32, 16)","{'alpha': 0.003, 'hidden_layer_sizes': (128, 6...",0.815949,0.817863,0.816906,0.817225,0.811164,0.815821,0.002409,20


In [8]:
cv_results.sort_values(['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,47.814530,11.446398,0.010743,0.000163,0.002,"(20, 10)","{'alpha': 0.002, 'hidden_layer_sizes': (20, 10)}",0.818182,0.829027,0.827113,0.822329,0.818820,0.823094,0.004344,1
2,63.556352,12.139283,0.038616,0.002949,0.001,"(128, 64, 32)","{'alpha': 0.001, 'hidden_layer_sizes': (128, 6...",0.817544,0.830303,0.822329,0.823923,0.820734,0.822967,0.004232,2
4,129.356023,21.904520,0.039742,0.001457,0.001,"(128, 64, 32, 16)","{'alpha': 0.001, 'hidden_layer_sizes': (128, 6...",0.818501,0.826475,0.827751,0.817863,0.823604,0.822839,0.004038,3
12,64.730712,12.083789,0.034171,0.000641,0.002,"(128, 64, 32)","{'alpha': 0.002, 'hidden_layer_sizes': (128, 6...",0.818501,0.829984,0.822010,0.823923,0.819458,0.822775,0.004078,4
0,60.139355,9.023936,0.017874,0.000682,0.001,"(20, 10)","{'alpha': 0.001, 'hidden_layer_sizes': (20, 10)}",0.817225,0.824242,0.827432,0.822010,0.819139,0.822010,0.003620,5
7,65.288633,11.542435,0.036058,0.001613,0.003,"(128, 64, 32)","{'alpha': 0.003, 'hidden_layer_sizes': (128, 6...",0.817225,0.828070,0.820734,0.823923,0.817225,0.821435,0.004151,6
17,57.352203,16.736445,0.036526,0.004243,0.005,"(128, 64, 32)","{'alpha': 0.005, 'hidden_layer_sizes': (128, 6...",0.818182,0.829665,0.819139,0.823923,0.814992,0.821180,0.005118,7
23,66.318924,9.101980,0.022085,0.001739,0.007,"(63, 32, 16, 8)","{'alpha': 0.007, 'hidden_layer_sizes': (63, 32...",0.806699,0.830303,0.825199,0.824880,0.817225,0.820861,0.008221,8
29,101.562784,24.672788,0.040807,0.000799,0.009,"(128, 64, 32, 16)","{'alpha': 0.009, 'hidden_layer_sizes': (128, 6...",0.815311,0.829027,0.819777,0.822967,0.810845,0.819585,0.006249,9
22,58.091176,16.907148,0.035333,0.002407,0.007,"(128, 64, 32)","{'alpha': 0.007, 'hidden_layer_sizes': (128, 6...",0.806380,0.829665,0.819777,0.824242,0.815630,0.819139,0.007904,10


In [9]:
optimizer.best_params_

{'alpha': 0.002, 'hidden_layer_sizes': (20, 10)}